In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pybaseball as pyb

In [3]:
# Define font properties for general text
font_properties = {'family': 'DejaVu Sans', 'size': 12}

# Define font properties for titles
font_properties_titles = {'family': 'DejaVu Sans', 'size': 20}

# Define font properties for axes labels
font_properties_axes = {'family': 'DejaVu Sans', 'size': 16}

# Set the theme for seaborn plots
sns.set_theme(style='whitegrid', 
              palette='deep', 
              font='DejaVu Sans', 
              font_scale=1.5, 
              color_codes=True, 
              rc=None)

# Import matplotlib
import matplotlib as mpl

# Set the resolution of the figures to 300 DPI
mpl.rcParams['figure.dpi'] = 300

In [7]:
print(pyb.playerid_lookup('Thomas','Lane'))

  name_last name_first  key_mlbam key_retro  key_bbref  key_fangraphs  \
0    thomas       lane     657041  thoml002  thomala02          16939   

   mlb_played_first  mlb_played_last  
0            2019.0           2024.0  


In [10]:
print(pyb.playerid_lookup('Thomas','Lane'))
batter_id = 657041
df_pyb = pyb.statcast_batter('2024-03-28', '2024-10-01', batter_id)
df_pyb.head()

  name_last name_first  key_mlbam key_retro  key_bbref  key_fangraphs  \
0    thomas       lane     657041  thoml002  thomala02          16939   

   mlb_played_first  mlb_played_last  
0            2019.0           2024.0  
Gathering Player Data


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle
0,CH,2024-09-28,82.5,-2.12,6.89,"Thomas, Lane",657041,434378,field_out,hit_into_play,...,3,2,8.0,1.0,NaN,7,2.73,1.07,1.07,47.8
1,CH,2024-09-28,83.0,-2.03,6.88,"Thomas, Lane",657041,434378,NaN,ball,...,3,2,8.0,1.0,NaN,7,2.58,1.04,1.04,48.1
2,SL,2024-09-28,88.2,-1.60,7.01,"Thomas, Lane",657041,434378,field_out,hit_into_play,...,2,1,8.0,1.0,NaN,7,2.41,-0.59,-0.59,55.1
3,SL,2024-09-28,88.5,-1.78,6.87,"Thomas, Lane",657041,434378,NaN,ball,...,2,1,8.0,1.0,NaN,7,2.40,-0.57,-0.57,58.0
4,CU,2024-09-28,78.7,-1.70,6.96,"Thomas, Lane",657041,434378,NaN,foul,...,2,1,8.0,1.0,NaN,7,4.88,-0.62,-0.62,58.6


In [ ]:
#i am very happy